In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ccfj
import h5py
import time
import os
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import sys
sys.path.append(r'../')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
proj_name = 'output_FJSJ_09-02/'
nThreads = 20

In [ ]:
filename_basic = proj_name+'Basic_info.npy'
info_basic = np.load(filename_basic, allow_pickle='TRUE').item()      # setting dictionary

### Define how many subworks to be done

In [ ]:
key_subworks = ['01-01']
key_subworks = info_basic['key_subworks']
#key_subworks.remove('2013_10')
key_subworks

### F-J

In [ ]:
def noise_fj(key_subwork):
    global proj_name
    global c

    start0 = time.time()

    filename = proj_name+'Basic_info.npy'
    info_basic = np.load(filename, allow_pickle='TRUE').item()
    
    dir_stack = info_basic['dir_stack']
    dir_ds = info_basic['dir_ds']
    f = info_basic['f']

    if os.path.exists(dir_ds+'ds_'+key_subwork+'.h5'):
        os.remove(dir_ds+'ds_'+key_subwork+'.h5')
    h5file = h5py.File(dir_ds+'ds_'+key_subwork+'.h5','w')

    #print("F-J scan for "+filename+" stack  "+key_subwork)
    """
    data = np.load(dir_stack+key_subwork+'_summed-'+filename+'.npz')
    ncfs = data['ncfs']
    r = data['r']*1e3
    """
    # linear stack
    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs = ncffile['ncfs'][:]
    r = ncffile['r'][:]*1e3
    ncffile.close()

    """
    # FPS stack
    outname = key_subwork+'_gather_FPS.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_FPS = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    ncffile.close()
    """


    #ds00 = ccfj.fj_noise(np.real(ncfs),r,c,f,fstride=1,itype=0,func=0)
    #ds01 = ccfj.fj_noise(np.real(ncfs),r,c,f,fstride=1,itype=1,func=0)
    #ds10 = ccfj.fj_noise(np.real(ncfs),r,c,f,fstride=1,itype=0,func=1)
    ds11 = ccfj.fj_noise(np.real(ncfs),r,c,f,fstride=1,itype=1,func=1)
    #ds = [ds10,ds11]
    #ds = np.array(ds)
    ds = np.array(ds11).reshape(1,np.shape(ds11)[0],np.shape(ds11)[1])
    h5file.create_dataset('ds_linear',data=ds)

    """
    ds11 = ccfj.fj_noise(np.real(ncfs_FPS),r,c,f,fstride=1,itype=1,func=1)
    ds = np.array(ds11).reshape(1,np.shape(ds11)[0],np.shape(ds11)[1])
    h5file.create_dataset('ds_FPS',data=ds)
    """

    h5file.create_dataset('f',data=f)
    h5file.create_dataset('c',data=c)
    h5file.close()
    print('Finish '+ key_subwork +   ' time:', time.time()-start0, ' seconds')

In [ ]:
c = np.linspace(0.100,2,1000)
info_basic['c'] = c

In [ ]:
"""
pool = ThreadPoolExecutor(max_workers = nThreads)
for key_subwork in key_subworks:
    pool.submit(noise_fj,key_subwork)
pool.shutdown()
"""

In [ ]:
for key_subwork in key_subworks:
    noise_fj(key_subwork)

In [ ]:
np.save(filename_basic,info_basic)